<a href="https://colab.research.google.com/github/ankitsaha392/Balloon-Game/blob/main/Copy_of_FastText_%2B_BiLSTM_%2B_Attention_traditional_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pas 2 algorithm

In [1]:
!pip install fasttext
import nltk
nltk.download('punkt_tab')
!pip install fasttext


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
import fasttext.util

nltk.download("punkt")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Load dataset
df = pd.read_excel("/content/sample_data/Final PAS.xlsx")


# Preprocess text
def preprocess_text(text):
    if isinstance(text, str):
        text = re.sub(r'[^ঀ-৿\s]', '', text)  # Keep only Bengali characters
        tokens = word_tokenize(text)
        return " ".join(tokens)
    return ""

df["processed_sentences"] = df["Bengali Sentence"].apply(preprocess_text)

# Convert labels to numerical values
label_encoder = LabelEncoder()
df["sense_encoded"] = label_encoder.fit_transform(df["Sense"])

# Tokenization and Padding
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df["processed_sentences"])
X_sequences = tokenizer.texts_to_sequences(df["processed_sentences"])
X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_sequences, padding='post')

y = np.array(df["sense_encoded"])

# Load Pretrained Bengali FastText Embeddings
fasttext.util.download_model('bn', if_exists='ignore')
ft = fasttext.load_model('cc.bn.300.bin')

embedding_dim = 300
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in ft:
        embedding_matrix[i] = ft[word]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Build a deep learning model with Transformer-based Attention
inputs = Input(shape=(X_padded.shape[1],))
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=X_padded.shape[1], trainable=False)(inputs)
bilstm = Bidirectional(LSTM(256, return_sequences=True))(embedding)

# Transformer-based Attention Layer
multihead_attention = MultiHeadAttention(num_heads=4, key_dim=embedding_dim)(bilstm, bilstm)
norm_attention = LayerNormalization(epsilon=1e-6)(multihead_attention + bilstm)
global_pooling = GlobalAveragePooling1D()(norm_attention)

dense1 = Dense(128, activation='relu')(global_pooling)
dropout = Dropout(0.5)(dense1)
outputs = Dense(len(df["Sense"].unique()), activation='softmax')(dropout)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with more epochs
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f"Context-Aware BoW Accuracy: {accuracy:.2f}")
labels_in_test = np.unique(y_test)  # Get only the labels present in y_test
print(classification_report(y_test, y_pred, labels=labels_in_test, target_names=label_encoder.classes_[labels_in_test]))



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - accuracy: 0.0043 - loss: 5.9069 - val_accuracy: 0.0070 - val_loss: 5.8342
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.0062 - loss: 5.8261 - val_accuracy: 0.0070 - val_loss: 5.7873
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.0162 - loss: 5.7439 - val_accuracy: 0.0400 - val_loss: 5.6122
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.0321 - loss: 5.5158 - val_accuracy: 0.0821 - val_loss: 5.2617
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.0810 - loss: 5.0990 - val_accuracy: 0.1411 - val_loss: 4.7663
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.1205 - loss: 4.6893 - val_accuracy: 0.2102 - val_loss: 4.4400
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.1911 - loss: 4.2004 - val_accuracy: 0.2573 - val_loss: 4.0755
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.2514 - loss: 3.7447 - val_acc

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=15dbb57256ce53e661bba828db42be5fe61000a6786caf08906e62182efa30b3
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfra

In [ ]:
print(df.columns)
!pip install deep-translator


Index(['Word ', 'Bengali Sentence', 'Sense', 'Replaced_word', 'LLM(ChatGPT)',
       'processed_sentences', 'sense_encoded'],
      dtype='object')
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.3 MB/s eta 0:00:00


In [ ]:
from deep_translator import GoogleTranslator
import pandas as pd
import numpy as np
import tensorflow as tf
from nltk.tokenize import word_tokenize
from collections import defaultdict
import time

# Create a defaultdict to store lists of values
sense_dict = defaultdict(list)

# Populate the dictionary from df
for key, value in zip(df['Sense'], df['Replaced_word']):
    sense_dict[key].append(value)

# Optional: convert to regular dict if needed
sense_dict = dict(sense_dict)

# Initialize Google Translator from deep-translator
translator = GoogleTranslator(source='bn', target='en')

# Function to replace polysemous word with mapped word from dictionary
def replace_polysemous_word(sentence):
    processed = preprocess_text(sentence)
    sequence = tokenizer.texts_to_sequences([processed])
    padded = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=X_padded.shape[1], padding='post')
    prediction = np.argmax(model.predict(padded), axis=1)[0]
    predicted_sense = label_encoder.inverse_transform([prediction])[0]

    input_tokens = word_tokenize(sentence)
    polysemous_words = df["Word "].unique().tolist()

    poly_word = None
    replaced = False
    output_tokens = []
    output_tokens_sense_version = []

    for token in input_tokens:
        if not replaced and token in polysemous_words:
            poly_word = token
            value = sense_dict.get(predicted_sense, predicted_sense)
            output_tokens.append(value[0] if isinstance(value, list) else value)
            output_tokens_sense_version.append(predicted_sense)
            replaced = True
        else:
            output_tokens.append(token)
            output_tokens_sense_version.append(token)

    modified_sentence = " ".join(output_tokens)
    sense_substituted_sentence = " ".join(output_tokens_sense_version)

    if not replaced:
        return modified_sentence, predicted_sense, poly_word, sentence

    return modified_sentence, predicted_sense, poly_word, sense_substituted_sentence

# Output list
results = []

# Loop through all sentences
for sentence in df["Bengali Sentence"]:
    user_input = sentence

    modified_sentence, sense, poly_word, sense_sentence = replace_polysemous_word(user_input)

    # Translate original and modified sentences using deep-translator
    translation_original = translator.translate(user_input)
    translation_modified = translator.translate(modified_sentence)

    results.append({
        "Original Bengali Sentence": translation_original,
        "English Translation": translation_modified
    })

    # time.sleep(1)  # optional: avoid rate limit if needed

# Save to CSV
pd.DataFrame(results).to_csv("translated_output.csv", index=False, encoding="utf-8-sig")

print("✅ File 'translated_output.csv' created.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━

In [ ]:
def replace_polysemous_word(sentence):
    processed = preprocess_text(sentence)
    sequence = tokenizer.texts_to_sequences([processed])
    padded = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=X_padded.shape[1], padding='post')
    prediction = np.argmax(model.predict(padded), axis=1)[0]
    predicted_sense = label_encoder.inverse_transform([prediction])[0]

    # 🆕 Get replacement word from Excel where sense matches
    matched_row = excel_df[excel_df['D'] == predicted_sense]  # Replace 'D' with actual sense column header
    if not matched_row.empty:
        replacement_word = matched_row.iloc[0]['E']  # Replace 'E' with your target word column header
    else:
        replacement_word = predicted_sense  # Fallback

    input_tokens = word_tokenize(sentence)
    polysemous_words = df["Word"].unique().tolist()

    poly_word = None
    replaced = False
    output_tokens = []

    for token in input_tokens:
        if not replaced and token in polysemous_words:
            poly_word = token
            output_tokens.append(replacement_word)  # 🆕 use mapped word
            replaced = True
        else:
            output_tokens.append(token)

    modified_sentence = " ".join(output_tokens)

    if not replaced:
        return modified_sentence, None, None

    return modified_sentence, predicted_sense, poly_word


In [ ]:
import re
import numpy as np
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from googletrans import Translator

# Load translator
translator = Translator()

# Load polysemous words list and model-related resources
polysemous_words = df["Word"].unique().tolist()     # Your DataFrame of polysemous words
max_len = X_padded.shape[1]                         # Same padding length used during training

# Bengali text preprocessing
def preprocess_text(text):
    # Remove punctuation and symbols (keep Bengali chars and whitespace)
    text = re.sub(r'[^ঀ-৿\s]', '', text)
    return text.lower().strip()

# Predict and replace all polysemous words in the sentence
def replace_all_polysemous_words(sentence):
    tokens = word_tokenize(sentence)
    processed = preprocess_text(sentence)
    modified_tokens = tokens.copy()

    detected_words = []
    predicted_senses = []

    for i, token in enumerate(tokens):
        if token in polysemous_words:
            # Replace only this instance for context-aware input
            temp_tokens = tokens.copy()
            temp_tokens[i] = "<target>"
            marked_sentence = " ".join(temp_tokens)
            marked_processed = preprocess_text(marked_sentence)

            # Predict sense using model
            sequence = tokenizer.texts_to_sequences([marked_processed])
            padded = pad_sequences(sequence, maxlen=max_len, padding='post')
            prediction = np.argmax(model.predict(padded), axis=1)[0]
            sense = label_encoder.inverse_transform([prediction])[0]

            # Replace original token with predicted sense
            modified_tokens[i] = sense
            detected_words.append(token)
            predicted_senses.append(sense)

    modified_sentence = " ".join(modified_tokens)
    return modified_sentence, detected_words, predicted_senses

# Interactive loop
while True:
    user_input = input("📝 Enter a Bengali sentence (or 'exit' to stop): ")
    if user_input.lower() == 'exit':
        break

    # Get modified sentence and sense info
    modified_sentence, words, senses = replace_all_polysemous_words(user_input)

    # Translate using Google Translate
    translated_original = translator.translate(user_input, src='bn', dest='en').text
    translated_modified = translator.translate(modified_sentence, src='bn', dest='en').text

    # Display results
    print("\n🔹 Original Bengali Sentence:")
    print(user_input)
    print("🔸 Translated (Original):", translated_original)

    print("\n🔹 Modified Sentence (Sense Substituted):")
    print(modified_sentence)
    print("🔸 Translated (Modified):", translated_modified)

    if words:
        print("\n🧠 Polysemous Words Detected and Replaced:")
        for w, s in zip(words, senses):
            print(f"   🔸 Word: {w} → ✅ Sense: {s}")
    else:
        print("\n🧠 No polysemous words detected.")

    print("-" * 60)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def predict_sense(sentence):
    # Preprocess the input
    sentence = preprocess_text(sentence)

    # Convert to sequences
    sequence = tokenizer.texts_to_sequences([sentence])

    # Pad the sequence
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=X_padded.shape[1], padding='post')

    # Get predictions
    prediction = np.argmax(model.predict(padded_sequence), axis=1)[0]

    # Decode the predicted sense
    predicted_sense = label_encoder.inverse_transform([prediction])[0]

    return predicted_sense

# Run inference
while True:
    user_input = input("Enter a Bengali sentence (or 'exit' to stop): ")
    if user_input.lower() == 'exit':
        break
    result = predict_sense(user_input)
    print(f"Predicted Sense: {result}")


In [ ]:
!pip install fasttext
import nltk
nltk.download('punkt_tab')
!pip install fasttext

In [ ]:
#!pip install fasttext nltk tensorflow  # Make sure all necessary libraries are installed
import nltk
#nltk.download('punkt_tab')

import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf # Import tensorflow only once at the beginning
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GRU, Conv1D, GlobalMaxPooling1D, SimpleRNN, Input, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
import fasttext.util
import fasttext

#nltk.download("punkt")

# Load dataset
df = pd.read_excel("/content/sample_data/pas.xlsx")

# Preprocess text
def preprocess_text(text):
    if isinstance(text, str):
        text = re.sub(r'[^ঀ-৿\s]', '', text)  # Keep only Bengali characters
        tokens = word_tokenize(text)
        return " ".join(tokens)
    return ""

df["processed_sentences"] = df["Bengali Sentence"].apply(preprocess_text)

# Encode labels
label_encoder = LabelEncoder()
df["sense_encoded"] = label_encoder.fit_transform(df["Sense"])

# Tokenization & Padding
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df["processed_sentences"])
X_sequences = tokenizer.texts_to_sequences(df["processed_sentences"])
X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_sequences, padding='post')
y = np.array(df["sense_encoded"])

# Load FastText
fasttext.util.download_model('bn', if_exists='ignore')
ft = fasttext.load_model('cc.bn.300.bin')

embedding_dim = 300
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in ft:
        embedding_matrix[i] = ft[word]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Common Embedding Layer
def get_embedding_layer(inputs):
    return Embedding(input_dim=vocab_size,
                     output_dim=embedding_dim,
                     weights=[embedding_matrix],
                     input_length=X_padded.shape[1],
                     trainable=False)(inputs)

# Model Builders
def build_bilstm_model():
    inputs = Input(shape=(X_padded.shape[1],))
    embedding = get_embedding_layer(inputs)
    x = Bidirectional(LSTM(128))(embedding)
    x = Dropout(0.5)(x)
    outputs = Dense(len(np.unique(y)), activation='softmax')(x)
    return Model(inputs, outputs)

def build_gru_model():
    inputs = Input(shape=(X_padded.shape[1],))
    embedding = get_embedding_layer(inputs)
    x = Bidirectional(GRU(128))(embedding)
    x = Dropout(0.5)(x)
    outputs = Dense(len(np.unique(y)), activation='softmax')(x)
    return Model(inputs, outputs)

def build_cnn_model():
    inputs = Input(shape=(X_padded.shape[1],))
    embedding = get_embedding_layer(inputs)
    x = Conv1D(128, 3, activation='relu')(embedding)
    x = GlobalMaxPooling1D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(len(np.unique(y)), activation='softmax')(x)
    return Model(inputs, outputs)

def build_rnn_model():
    inputs = Input(shape=(X_padded.shape[1],))
    embedding = get_embedding_layer(inputs)
    x = SimpleRNN(128)(embedding)
    x = Dropout(0.5)(x)
    outputs = Dense(len(np.unique(y)), activation='softmax')(x)
    return Model(inputs, outputs)

def build_transformer_model():
    # Build a deep learning model with Transformer-based Attention
    inputs = Input(shape=(X_padded.shape[1],))
    embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=X_padded.shape[1], trainable=False)(inputs)
    bilstm = Bidirectional(LSTM(256, return_sequences=True))(embedding)

# Transformer-based Attention Layer
    multihead_attention = MultiHeadAttention(num_heads=4, key_dim=embedding_dim)(bilstm, bilstm)
    norm_attention = LayerNormalization(epsilon=1e-6)(multihead_attention + bilstm)
    global_pooling = GlobalAveragePooling1D()(norm_attention)

    dense1 = Dense(128, activation='relu')(global_pooling)
    dropout = Dropout(0.5)(dense1)
    outputs = Dense(len(df["Sense"].unique()), activation='softmax')(dropout)
    return Model(inputs, outputs)

# List of models to train
model_builders = {
    "BiLSTM": build_bilstm_model,
    "GRU": build_gru_model,
    "CNN": build_cnn_model,
    "RNN": build_rnn_model,
    "Transformer-Based": build_transformer_model
}

# Training & Evaluation
for name, builder in model_builders.items():
    print(f"\nTraining {name} model...")
    model = builder()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=0)

    y_pred = np.argmax(model.predict(X_test), axis=1)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {acc:.2f}")
    labels_in_test = np.unique(y_test)
    print(classification_report(y_test, y_pred, labels=labels_in_test, target_names=label_encoder.classes_[labels_in_test]))
